**Section 4: Synthetic Matrix Performance**

*Imports*

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import timeit
import math

from archive.randomized_projected_nmf import *
from random_matrix import *
from nmf import * 
from benchmark import *
from collections import defaultdict

*Benchmark*

In [2]:
methods = {
    "MU C": nmf_compress_mu,
    'MU SC': nmf_structured_compress_mu,
    'HALS C': nmf_compress_hals,
    'HALS SC': nmf_structured_compress_hals
}
methods_baseline = {
    "MU": nmf_mu,
    'HALS': nmf_hals,
}


projection_types = [
    'gaussian',
    'srht',
    'srft',
    'sparse-jl',
    'count-sketch',
]

rows = []

# Parameters
sizes = np.arange(1_000, 5_001, 1_000)
runs = 10
r = 20
for n in sizes:
    # Generate A (n x 0.75n)
    np.random.seed(1)
    m = int(0.75*n)
    for density, label in [(0.01,'sparse'),(1.0,'dense')]:
        A, _, _= generate_synthetic_matrix(n=m,m=n,r=r,delta = density)

        # Compressed Methods
        for method_name, method in methods.items():
            for projection in projection_types:
                total_times = [] 
                total_errors = []       
                for i in range(runs):
                    # Set seed per run
                    seed = i + 1
                    
                    # Time NMF Method
                    start_time = timeit.default_timer()
                    _, _, errors = method(A, r, random_state=seed,projection_type = projection)
                    time = timeit.default_timer() - start_time

                    # Store
                    total_times.append(time)
                    total_errors.append(errors[-1])
                
                rows.append(
                    {
                        'Method':method_name,
                        'Projection Type':projection,
                        'Size':n,
                        'Density':label,
                        'Time': np.mean(total_times),
                        'Errors': np.mean(total_errors)
                    }
                )
        # Regular Methods
        for method_name,method in methods_baseline.items():
            total_times = [] 
            total_errors = []       
            for i in range(runs):
                # Set seed per run
                seed = i + 1
                
                # Time NMF Method
                start_time = timeit.default_timer()
                _, _, errors = method(A, r)
                time = timeit.default_timer() - start_time

                # Store
                total_times.append(time)
                total_errors.append(errors[-1])
            
            rows.append(
                {
                    'Method':method_name,
                    'Projection Type':'',
                    'Size':n,
                    'Density':label,
                    'Time': np.mean(total_times),
                    'Errors': np.mean(total_errors)
                }
            )


    print(f'Completed benchmarking for matrix size: {n}')

Completed benchmarking for matrix size: 1000
Completed benchmarking for matrix size: 2000
Completed benchmarking for matrix size: 3000
Completed benchmarking for matrix size: 4000
Completed benchmarking for matrix size: 5000


In [7]:
stats = pd.DataFrame(rows)

In [8]:
stats['Method'].unique()

array(['MU C', 'MU SC', 'HALS C', 'HALS SC', 'MU', 'HALS'], dtype=object)

In [9]:
stats

,Method,Projection Type,Size,Density,Time,Errors
0,MU C,gaussian,1000,sparse,0.194602,0.709056
1,MU C,srht,1000,sparse,0.194047,0.633738
2,MU C,srft,1000,sparse,0.197744,0.733104
3,MU C,sparse-jl,1000,sparse,0.191821,0.775202
4,MU C,count-sketch,1000,sparse,0.190195,0.716401
...,...,...,...,...,...,...
215,HALS SC,srft,5000,dense,5.838787,0.200685
216,HALS SC,sparse-jl,5000,dense,5.872993,0.200471
217,HALS SC,count-sketch,5000,dense,5.878602,0.200588
218,MU,,5000,dense,10.358453,0.205798


*Plot: Computation Time by Method/Projection*

*i) dense*

In [17]:
stats['Projection Type'].unique()

array(['gaussian', 'srht', 'srft', 'sparse-jl', 'count-sketch', ''],
      dtype=object)

In [18]:
names = {
    'gaussian':'Gaussian',
    'srht':'SRHT',
    'srft':'SRFT',
    'sparse-jl':'Sparse JL',
    'count-sketch':'CountSketch'
}

In [38]:
dense = stats[(stats['Density'] == 'dense') & (stats['Projection Type'] != '')]

fig = make_subplots(rows = 1,cols = 4,shared_xaxes = True,shared_yaxes=True,subplot_titles=dense['Method'].unique())

methods = stats['Projection Type'].unique()
import plotly.express as px
colors = px.colors.qualitative.Plotly  # Plotly's default color sequence

# Create color mapping (cycles through colors if more methods than colors)
method_colors = {method: colors[i % len(colors)] for i, method in enumerate(methods)}


for i, method in enumerate(dense['Method'].unique()):
    for j, projection in enumerate(dense['Projection Type'].unique()):
        _df = dense[(dense['Method'] == method) & (dense['Projection Type'] == projection)]
        fig.add_trace(
            go.Scatter(
                x = _df['Size'],
                y = _df['Time'],
                name = names[projection],
                legendgroup=projection,  # Group by projection type
                line_color=method_colors[projection],  # Color by projection
                showlegend = (i == 0),
            ),
            row = 1, col = i + 1,
            
        )

mu_dense = stats[(stats['Method'] == 'MU') & (stats['Density'] == "dense")]

for i in range(2):
    fig.add_trace(
        go.Scatter(
            x = mu_dense['Size'],
            y = mu_dense['Time'],
            name = 'MU',
            showlegend = (i == 0),
            line = dict(color='black',dash='dash'),
            legendgroup = 'MU'
        ),
        row = 1,col= i+1
    )


hals_dense = stats[(stats['Method'] == 'HALS') & (stats['Density'] == "dense")]
for i in range(2,4):
    fig.add_trace(
        go.Scatter(
            x = hals_dense['Size'],
            y = hals_dense['Time'],
            name = 'HALS',
            showlegend = (i == 2),
            line = dict(color='grey',dash='dash'),
            legendgroup = 'HALS'
        ),
        row = 1,col= i+1
    )

fig.update_layout(
    xaxis1_title = 'Size (n)',
    yaxis1_title = 'Time (s)',
    xaxis2_title = 'Size (n)',
    yaxis2_title = 'Time (s)',
    xaxis3_title = 'Size (n)',
    yaxis3_title = 'Time (s)',
    xaxis4_title = 'Size (n)',
    yaxis4_title = 'Time (s)',
    template = 'plotly_white',
    legend=dict(
        orientation="h",  # horizontal
        y=-0.2,           # move legend below the plot
        x=0.5,
        xanchor="center",
        yanchor="top"
    ),
    font=dict(
        size=10,color='black'
    ),
    margin=dict(
        l=20,  # left margin
        r=20,  # right margin
        t=40,  # top margin
        b=40   # bottom margin
    ), 
    width = 800,
    height = 400
)

fig.show()

*ii) sparse*

In [32]:
sparse = stats[(stats['Density'] == 'sparse') & (stats['Projection Type'] != '')]

fig = make_subplots(rows = 1,cols = 4,shared_xaxes = True,shared_yaxes=True,subplot_titles=sparse['Method'].unique())

for i, method in enumerate(sparse['Method'].unique()):
    for j, projection in enumerate(sparse['Projection Type'].unique()):
        _df = sparse[(sparse['Method'] == method) & (sparse['Projection Type'] == projection)]
        fig.add_trace(
            go.Scatter(
                x = _df['Size'],
                y = _df['Time'],
                name = names[projection],
                legendgroup=projection,  # Group by projection type
                line_color=method_colors[projection],  # Color by projection
                showlegend = (i == 0),
            ),
            row = 1, col = i + 1,
            
        )
mu_dense = stats[(stats['Method'] == 'MU') & (stats['Density'] == "sparse")]

for i in range(2):
    fig.add_trace(
        go.Scatter(
            x = mu_dense['Size'],
            y = mu_dense['Time'],
            name = 'MU',
            showlegend = (i == 0),
            line = dict(color='black',dash='dash'),
            legendgroup = 'MU'
        ),
        row = 1,col= i+1
    )


hals_dense = stats[(stats['Method'] == 'HALS') & (stats['Density'] == "sparse")]
for i in range(2,4):
    fig.add_trace(
        go.Scatter(
            x = hals_dense['Size'],
            y = hals_dense['Time'],
            name = 'HALS',
            showlegend = (i == 2),
            line = dict(color='grey',dash='dash'),
            legendgroup = 'HALS'
        ),
        row = 1,col= i+1
    )


fig.update_layout(
    xaxis1_title = 'Size (n)',
    yaxis1_title = 'Time (s)',
    xaxis2_title = 'Size (n)',
    yaxis2_title = 'Time (s)',
    xaxis3_title = 'Size (n)',
    yaxis3_title = 'Time (s)',
    xaxis4_title = 'Size (n)',
    yaxis4_title = 'Time (s)',
    template = 'plotly_white',
    legend=dict(
        orientation="h",  # horizontal
        y=-0.2,           # move legend below the plot
        x=0.5,
        xanchor="center",
        yanchor="top"
    ),
    font=dict(
        size=10,color='black'
    ),
    margin=dict(
        l=20,  # left margin
        r=20,  # right margin
        t=40,  # top margin
        b=40   # bottom margin
    ), 
    width = 800,
    height = 400
)

fig.show()

*Plot: Reconstruction Error by Method/Projection*

*i) dense*

In [34]:
dense = stats[(stats['Density'] == 'sparse') & (stats['Projection Type'] != '')]

fig = make_subplots(rows = 1,cols = 4,shared_xaxes = True,shared_yaxes=True,subplot_titles=dense['Method'].unique())


for i, method in enumerate(dense['Method'].unique()):
    for j, projection in enumerate(dense['Projection Type'].unique()):
        _df = dense[(dense['Method'] == method) & (dense['Projection Type'] == projection)]
        fig.add_trace(
            go.Scatter(
                x = _df['Size'],
                y = _df['Errors'],
                name = names[projection],
                legendgroup=projection,  # Group by projection type
                line_color=method_colors[projection],  # Color by projection
                showlegend = (i == 0),
            ),
            row = 1, col = i + 1,
            
        )

mu_dense = stats[(stats['Method'] == 'MU') & (stats['Density'] == "dense")]

for i in range(2):
    fig.add_trace(
        go.Scatter(
            x = mu_dense['Size'],
            y = mu_dense['Errors'],
            name = 'MU',
            showlegend = (i == 0),
            line = dict(color='black',dash='dash'),
            legendgroup = 'MU'
        ),
        row = 1,col= i+1
    )


hals_dense = stats[(stats['Method'] == 'HALS') & (stats['Density'] == "dense")]
for i in range(2,4):
    fig.add_trace(
        go.Scatter(
            x = hals_dense['Size'],
            y = hals_dense['Errors'],
            name = 'HALS',
            showlegend = (i == 2),
            line = dict(color='grey',dash='dash'),
            legendgroup = 'HALS'
        ),
        row = 1,col= i+1
    )


fig.update_layout(
    xaxis1_title = 'Size (n)',
    yaxis1_title = 'Error',
    xaxis2_title = 'Size (n)',
    yaxis2_title = 'Error',
    xaxis3_title = 'Size (n)',
    yaxis3_title = 'Error',
    xaxis4_title = 'Size (n)',
    yaxis4_title = 'Error',
    template = 'plotly_white',
    legend=dict(
        orientation="h",  # horizontal
        y=-0.2,           # move legend below the plot
        x=0.5,
        xanchor="center",
        yanchor="top"
    ),
    font=dict(
        size=10,color='black'
    ),
    margin=dict(
        l=20,  # left margin
        r=20,  # right margin
        t=40,  # top margin
        b=40   # bottom margin
    ), 
    width = 800,
    height = 400
)
fig.show()

*ii) sparse*

In [35]:
sparse = stats[(stats['Density'] == 'sparse') & (stats['Projection Type'] != '')]

fig = make_subplots(rows = 1,cols = 4,shared_xaxes = True,shared_yaxes=True,subplot_titles=sparse['Method'].unique())


for i, method in enumerate(sparse['Method'].unique()):
    for j, projection in enumerate(sparse['Projection Type'].unique()):
        _df = sparse[(sparse['Method'] == method) & (sparse['Projection Type'] == projection)]
        fig.add_trace(
            go.Scatter(
                x = _df['Size'],
                y = _df['Errors'],
                name = names[projection],
                legendgroup=projection,  # Group by projection type
                line_color=method_colors[projection],  # Color by projection
                showlegend = (i == 0),
            ),
            row = 1, col = i + 1,
            
        )
mu_sparse = stats[(stats['Method'] == 'MU') & (stats['Density'] == "sparse")]

for i in range(2):
    fig.add_trace(
        go.Scatter(
            x = mu_sparse['Size'],
            y = mu_sparse['Errors'],
            name = 'MU',
            showlegend = (i == 0),
            line = dict(color='black',dash='dash'),
            legendgroup = 'MU'
        ),
        row = 1,col= i+1
    )


hals_sparse = stats[(stats['Method'] == 'HALS') & (stats['Density'] == "sparse")]
for i in range(2,4):
    fig.add_trace(
        go.Scatter(
            x = hals_sparse['Size'],
            y = hals_sparse['Errors'],
            name = 'HALS',
            showlegend = (i == 2),
            line = dict(color='grey',dash='dash'),
            legendgroup = 'HALS'
        ),
        row = 1,col= i+1
    )


fig.update_layout(
    xaxis1_title = 'Size (n)',
    yaxis1_title = 'Error',
    xaxis2_title = 'Size (n)',
    yaxis2_title = 'Error',
    xaxis3_title = 'Size (n)',
    yaxis3_title = 'Error',
    xaxis4_title = 'Size (n)',
    yaxis4_title = 'Error',
    template = 'plotly_white',
    legend=dict(
        orientation="h",  # horizontal
        y=-0.2,           # move legend below the plot
        x=0.5,
        xanchor="center",
        yanchor="top"
    ),
    font=dict(
        size=10,color='black'
    ),
    margin=dict(
        l=20,  # left margin
        r=20,  # right margin
        t=40,  # top margin
        b=40   # bottom margin
    ), 
    width = 800,
    height = 400
)

fig.show()

In [79]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Define names mapping
names = {
    'gaussian':'Gaussian',
    'srht':'SRHT',
    'srft':'SRFT',
    'sparse-jl':'Sparse JL',
    'count-sketch':'CountSketch'
}

# Define the exact order of methods we want
methods = ['MU C', 'MU SC', 'HALS C', 'HALS SC']  # Explicit order

# Get non-empty projections
projections = [p for p in stats['Projection Type'].unique() if p != '']

# Create color mapping
colors = px.colors.qualitative.Plotly
method_colors = {projection: colors[i % len(colors)] for i, projection in enumerate(projections)}

# Create figure with 2 rows and 4 columns
fig = make_subplots(
    rows=2, cols=4,
    shared_xaxes=True,
    shared_yaxes='rows',
    subplot_titles=methods*2,  # Same order for both rows
    horizontal_spacing=0.05,
    vertical_spacing=0.08
)

# Add Time plots (row 1)
for col, method in enumerate(methods, 1):
    # Filter for current method, dense data, and non-empty projections
    dense = stats[(stats['Method'] == method) & (stats['Density'] == 'dense') & (stats['Projection Type'] != '')]
    
    # Add projection methods
    for projection in projections:
        _df = dense[dense['Projection Type'] == projection]
        if not _df.empty:
            fig.add_trace(
                go.Scatter(
                    x=_df['Size'],
                    y=_df['Time'],
                    name=names.get(projection, projection),
                    legendgroup=projection,
                    line_color=method_colors[projection],
                    showlegend=(col == 1),
                    mode='lines+markers'
                ),
                row=1, col=col
            )
    
    # Add baseline methods - CORRECTED TO FILTER FOR EMPTY PROJECTION TYPE
    if method in ['MU C', 'MU SC']:
        baseline = stats[(stats['Method'] == 'MU') & (stats['Density'] == "dense") & (stats['Projection Type'] == '')]
        if not baseline.empty:
            fig.add_trace(
                go.Scatter(
                    x=baseline['Size'],
                    y=baseline['Time'],
                    name='MU',
                    showlegend=(col == 1),
                    line=dict(color='black', dash='dash'),
                    legendgroup='MU'
                ),
                row=1, col=col
            )
    elif method in ['HALS C', 'HALS SC']:
        baseline = stats[(stats['Method'] == 'HALS') & (stats['Density'] == "dense") & (stats['Projection Type'] == '')]
        if not baseline.empty:
            fig.add_trace(
                go.Scatter(
                    x=baseline['Size'],
                    y=baseline['Time'],
                    name='HALS',
                    showlegend=(col == 3),  # Show for first HALS column
                    line=dict(color='grey', dash='dash'),
                    legendgroup='HALS'
                ),
                row=1, col=col
            )

# Add Error plots (row 2)
for col, method in enumerate(methods, 1):
    # Filter for current method, dense data, and non-empty projections
    dense = stats[(stats['Method'] == method) & (stats['Density'] == 'dense') & (stats['Projection Type'] != '')]
    
    for projection in projections:
        _df = dense[dense['Projection Type'] == projection]
        if not _df.empty:
            fig.add_trace(
                go.Scatter(
                    x=_df['Size'],
                    y=_df['Errors'],
                    name=names.get(projection, projection),
                    legendgroup=projection,
                    line_color=method_colors[projection],
                    showlegend=False,
                    mode='lines+markers'
                ),
                row=2, col=col
            )
    
    # Add baseline methods for error plots - CORRECTED TO FILTER FOR EMPTY PROJECTION TYPE
    if method in ['MU C', 'MU SC']:
        baseline = stats[(stats['Method'] == 'MU') & (stats['Density'] == "dense") & (stats['Projection Type'] == '')]
        if not baseline.empty:
            fig.add_trace(
                go.Scatter(
                    x=baseline['Size'],
                    y=baseline['Errors'],
                    name='MU',
                    showlegend=False,
                    line=dict(color='black', dash='dash'),
                    legendgroup='MU'
                ),
                row=2, col=col
            )
    elif method in ['HALS C', 'HALS SC']:
        baseline = stats[(stats['Method'] == 'HALS') & (stats['Density'] == "dense") & (stats['Projection Type'] == '')]
        if not baseline.empty:
            fig.add_trace(
                go.Scatter(
                    x=baseline['Size'],
                    y=baseline['Errors'],
                    name='HALS',
                    showlegend=False,
                    line=dict(color='grey', dash='dash'),
                    legendgroup='HALS'
                ),
                row=2, col=col
            )

# Update layout
fig.update_layout(
    template='plotly_white+simple_white',
    legend=dict(
        orientation="h",
        y=-0.2,
        x=0.5,
        xanchor="center",
        yanchor="top"
    ),
    font=dict(size=12, color='black'),
    margin=dict(l=40, r=40, t=60, b=100),
    width=1200,
    height=600
)

# Update axis titles
for col in range(1, 5):
    fig.update_xaxes(title_text="Size (n)", row=2, col=col)
    fig.update_yaxes(title_text="Time (s)", row=1, col=col)
    fig.update_yaxes(title_text="Error", row=2, col=col)
    
for i in range(1, 9):  # For 8 axes (2 rows × 4 columns)
    fig.update_xaxes(showgrid=True, gridcolor='lightgray', row=(i-1)//4+1, col=(i-1)%4+1)
    fig.update_yaxes(showgrid=True, gridcolor='lightgray', row=(i-1)//4+1, col=(i-1)%4+1)
fig.show()

In [78]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Define names mapping
names = {
    'gaussian':'Gaussian',
    'srht':'SRHT',
    'srft':'SRFT',
    'sparse-jl':'Sparse JL',
    'count-sketch':'CountSketch'
}

# Define the exact order of methods we want
methods = ['MU C', 'MU SC', 'HALS C', 'HALS SC']  # Explicit order

# Get non-empty projections
projections = [p for p in stats['Projection Type'].unique() if p != '']

# Create color mapping
colors = px.colors.qualitative.Plotly
method_colors = {projection: colors[i % len(colors)] for i, projection in enumerate(projections)}

# Create figure with 2 rows and 4 columns
fig = make_subplots(
    rows=2, cols=4,
    shared_xaxes=True,
    shared_yaxes='rows',
    subplot_titles=methods*2,  # Same order for both rows
    horizontal_spacing=0.05,
    vertical_spacing=0.08
)

# Add Time plots (row 1) - SPARSE VERSION
for col, method in enumerate(methods, 1):
    # Filter for current method, SPARSE data, and non-empty projections
    sparse = stats[(stats['Method'] == method) & (stats['Density'] == 'sparse') & (stats['Projection Type'] != '')]
    
    # Add projection methods
    for projection in projections:
        _df = sparse[sparse['Projection Type'] == projection]
        if not _df.empty:
            fig.add_trace(
                go.Scatter(
                    x=_df['Size'],
                    y=_df['Time'],
                    name=names.get(projection, projection),
                    legendgroup=projection,
                    line_color=method_colors[projection],
                    showlegend=(col == 1),
                    mode='lines+markers'
                ),
                row=1, col=col
            )
    
    # Add baseline methods - SPARSE VERSION with empty projection type
    if method in ['MU C', 'MU SC']:
        baseline = stats[(stats['Method'] == 'MU') & (stats['Density'] == "sparse") & (stats['Projection Type'] == '')]
        if not baseline.empty:
            fig.add_trace(
                go.Scatter(
                    x=baseline['Size'],
                    y=baseline['Time'],
                    name='MU',
                    showlegend=(col == 1),
                    line=dict(color='black', dash='dash'),
                    legendgroup='MU'
                ),
                row=1, col=col
            )
    elif method in ['HALS C', 'HALS SC']:
        baseline = stats[(stats['Method'] == 'HALS') & (stats['Density'] == "sparse") & (stats['Projection Type'] == '')]
        if not baseline.empty:
            fig.add_trace(
                go.Scatter(
                    x=baseline['Size'],
                    y=baseline['Time'],
                    name='HALS',
                    showlegend=(col == 3),  # Show for first HALS column
                    line=dict(color='grey', dash='dash'),
                    legendgroup='HALS'
                ),
                row=1, col=col
            )

# Add Error plots (row 2) - SPARSE VERSION
for col, method in enumerate(methods, 1):
    # Filter for current method, SPARSE data, and non-empty projections
    sparse = stats[(stats['Method'] == method) & (stats['Density'] == 'sparse') & (stats['Projection Type'] != '')]
    
    for projection in projections:
        _df = sparse[sparse['Projection Type'] == projection]
        if not _df.empty:
            fig.add_trace(
                go.Scatter(
                    x=_df['Size'],
                    y=_df['Errors'],
                    name=names.get(projection, projection),
                    legendgroup=projection,
                    line_color=method_colors[projection],
                    showlegend=False,
                    mode='lines+markers'
                ),
                row=2, col=col
            )
    
    # Add baseline methods for error plots - SPARSE VERSION with empty projection type
    if method in ['MU C', 'MU SC']:
        baseline = stats[(stats['Method'] == 'MU') & (stats['Density'] == "sparse") & (stats['Projection Type'] == '')]
        if not baseline.empty:
            fig.add_trace(
                go.Scatter(
                    x=baseline['Size'],
                    y=baseline['Errors'],
                    name='MU',
                    showlegend=False,
                    line=dict(color='black', dash='dash'),
                    legendgroup='MU'
                ),
                row=2, col=col
            )
    elif method in ['HALS C', 'HALS SC']:
        baseline = stats[(stats['Method'] == 'HALS') & (stats['Density'] == "sparse") & (stats['Projection Type'] == '')]
        if not baseline.empty:
            fig.add_trace(
                go.Scatter(
                    x=baseline['Size'],
                    y=baseline['Errors'],
                    name='HALS',
                    showlegend=False,
                    line=dict(color='grey', dash='dash'),
                    legendgroup='HALS'
                ),
                row=2, col=col
            )

# Update layout
fig.update_layout(
    template='plotly_white+simple_white',
    legend=dict(
        orientation="h",
        y=-0.2,
        x=0.5,
        xanchor="center",
        yanchor="top"
    ),
    font=dict(size=12, color='black'),
    margin=dict(l=40, r=40, t=60, b=100),
    width=1200,
    height=600
)

# Update axis titles
for col in range(1, 5):
    fig.update_xaxes(title_text="Size (n)", row=2, col=col)
    fig.update_yaxes(title_text="Time (s)", row=1, col=col)
    fig.update_yaxes(title_text="Error", row=2, col=col)
    
for i in range(1, 9):  # For 8 axes (2 rows × 4 columns)
    fig.update_xaxes(showgrid=True, gridcolor='lightgray', row=(i-1)//4+1, col=(i-1)%4+1)
    fig.update_yaxes(showgrid=True, gridcolor='lightgray', row=(i-1)//4+1, col=(i-1)%4+1)
fig.show()